<a href="https://colab.research.google.com/github/abernauer/Deep-Learning-with-Python/blob/master/Chapter_4Fundamentalsofmachinelearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Central problem of machine learning: overfitting.

#4.1 Four branches of machine learning

In the previous examples, we covered: binary classification, multiclass classification, and scalar regression. All three fall under *supervised learning*, where the goal is to learn the relationship between training inputs and training targets.

* consider going back and using TeX for math expressions

#4.1.1 Supervised learning 

Supervised learning consists of learning to map input data to known targets(also called *annotations*), given a set of examples.

Although supervised learning mostly consists of classification and regression, there are more exotic variants as well:

* Sequence generation -- Given a picture, predict a caption describing it. Sequence generation can sometimes be reformulated as a series of classification (such as repeatedly predicting a word or token in a sequence).
* *Syntax tree prediction* -- Given a sentence , predict its decomposition into a syntax tree.
* *Object detection*--Given a picture, draw a bounding box around certain objects inside the picture. This can also be expressed as a classification problem(given many candidate bounding boxes, classify the contents of each one) or as a joint classification and regression problem, where the bounding-box coordinates are predicted via vector regression.
* *Image segmentation*--Given a picture, draw a pixel-level mask on a specific object.

#4.1.2 Unsupervised learning

Unsupervised learning is the bread and butter of data analytics, and it's often a necessary step in better understanding a dataset before attempting to solve a supervised-learning problem. *Dimensionality reduction* and *clustering* are well-known categories of unsupervised learning.

#4.1.3 Self-supervised learning

Self-supervised learning is supervised learning without human-annotated labels--you can think of it as supervised learning without any humans in the loop. Labels are still involved, but they're generated from the input data, typically using a heuristic algorithm.

For instance, *autoencoders* are a well-known instance of self-supervised learning, where the generated targets are the input, unmodified. In the same way, trying to predict the next frame in a video, given past frames, or the next word in a text, given previous words, are instances of self-supervised learning. Note that the distinction between supervised, self-supervised, and unsupervised learning can be blurry sometimes-- these categories are more of a continuum withouth solid borders. 

#4.1.4 Reinforcement learning

In reinforcement learning, an *agent* receives information about its environment and learns to choose actions that will maximize some reward. For instance, a neural network that "looks" at a video game screen and outputs game actions in order to maximize its score can be trained via reinforcement learning.

#Classification and regression glossary

* Sample or input--One data point that goes into your model.
* Prediction or output-- What comes out of your model.
* Target -- The truth, What your model should ideally have predicted, according to an external source of data.
* Prediction error or loss value-- A measure of the distance between your model's prediction and the target.
* Classes-- A set of possible labels to choose from in a classification problem. For example, when classifying cat and dog pictures, "dog" and "cat" are the two classes.
* Ground-truth or annotations-- All targets for a dataset, typically collected by humans.
* Binary classification -- A classification task where each input sample should be categorized into two exclusive categories.
* Multiclass classification-- A classification task where each input sample should be categorized into more than two categories: for instance, classifying handwritten digits.
* Multilabel classification-- A classification task where each input sample can be assigned multiple labels. For instances, a given image may contain both a cat and a dog and should be annotated both with the "cat" label and the "dog" label. The number of labels per image is usually variable.
* Vector regression -- A task where the target is a set of continous values: for example, a continuous vector.
* Mini-batch or batch -- A small set of samples(typically between 8 and 128) that are processed simultaneously by the model.

#4.2 Evaluating machine-learning models

* First key problem in machine learning avoid *overfitting*. Or performing better on the training data and stalling on the test data after a few iterations or epochs.

* Second key problem *generalize* performance on to never before seen data.

#4.2.1 Training, validation, and test sets

1. Split the data into three sets: training, validation, and test.
2. Train on your training data and evaluate you model on the validation data.
3. Test the model on the test data.



The validation set allows us to tune the configuration of our model: choosing the number of layers or size of the layers( *hyperparameters*). Tuning the hyperparameters is a form of *learning*: a search problem for a good configuration in some parameter space. This can result in *overfitting on the validation set*.

*Information leaks* may take place if you optimize your hyperparameters too much on the validation set as you will expose information to the model that will tune it specifically for performance on the validation set. Not for the test set which we would like to perform well on.

# Simple Hold-out validation

Set apart some fraction of your data as your test set. Train on the remaining data, and evauate on the test set.

```
 import numpy as np
 from keras import models

 num_validation_samples = 10000

 np.random.shuffle(data)

 validation_data = data[:num_validation_samples]
 data = data[num_validation_samples:]

 training_data = data[:]

 model = get_model()
 model.train(training_data)
 validation_score = model.evaluate(validation_data)

 # At this point you can tune your model.
 # retrain it, evaluate it, tune it again.

 model = get_model()
 model.train(np.concatenate([training_data,
                             validation_data]))
 test_score = model.evaluate(test_data)

 ```

This is the simplest evaluation protocol, and it suffers from one flaw: if little data is available, then your validattion and test sets may contain too few samples to be statistically representative of the data at hand. 

# K-FOLD Validation

With this approach, you split your data into K partitions of equal size. For each partition i, train a model on the remaining K-1 partitions, and evaluate it on partition i. Your final score is then the averages of the K scores obtained. This method is helpful when the performance of your model shows significant variance based on your training test split. 

```
k = 4
num_validation_samples = len(data) // k

np.random.shuffle(data)

validation_scores = []
for fold in range(k):
    validation_data = data[num_validation_samples * fold:
    num_validation_samples * (fold + 1)]
    training_data = data[:num_validation_samples * fold] + 
    data[num_validation_samples * (fold + 1):]
    model = get_model()
    model.train(training_data)
    validation_score = model.evaluate(validation_data)
    validation_scores.append(validation_score)
validation_score = np.average(validation_scores)

model = get_model()
model.train(data)
test_score = model.evaluate(test_data)

```

#Iterated K-FOLD VALIDATION WITH SHUFFLING

It consists of applying K-fold validation multiple times, shuffing the data every time before splitting it K ways. The final score is the average of the scores obtained at each run of K-fold validation. Note that you end up training and evaluating P X K models(where P is the number of iterations you use), which can be very expensive.

#4.2.2 Things to keep in mind

* *Data representiveness*-- You want both your taining set and test set to be representative of the data at hand. For instance, if you're trying to classify images of digits, and you're starting from an array of samples where the samples are ordered by their class, takinge the first 80% of the array as your training set and the remaining 20% as your test set will result in your training set containing only classes 0-7, whereas your test set contatins only classes 8-9. This seems like a ridiculous mistake, but it's surprisingly common. For this reason, you usually should *randomly shuffle* your data before splitting it into training and test sets.

* *The arrow of time*-- If you're trying to predict the future given the past, you should not randomly shuffle your data before splitting it, because doing so will create a *temporal leak*: your model will effectively be trained on data from the future. In such situations you should always make sure all data in your test set *posterior* to the data in the training set.

* *Redundancy in your data*-- If some data points in your data appear twice, then shuffling the data and splitting it into a training set and validation set will result in redundacy between the training and validation sets. In effect, you'll be testing on part of your training data, which is the worst thing you can do! Make sure your training set and validation set are disjoint

#4.3 Data preprocessing, feature engineering, and feature learning

In addition to model evaluation, an important question we must must tackle before we dive deeper into model development is the following: how do you prepare the input data and targets before feeding them into a neural network?

#4.3.1 Data preprocessing for neural networks


#Vectorization

All inputs and targets in a neural network must be tensors of floating-point. Whatever data you need to process you must first turn into tensors, a step called *data vectorization*. 

#Value normalization 

In general, it isn't safe to feed into a neural network data that takes a relatively large values or data that is heterogeneous. Doing so can trigger large gradient updates that will prevent the network from converging. 

* *Take small values*-- Typically, most values should be in the 0-1 range.
* Be *homogenous*-- That is, all features should take values in roughly the same range.

* Normalize each feature independently to have a mean of 0.

* Normalize each feature independently to have a standard deviation of 1.

```
x -= x.mean(axis=0)
x /= x.std(axis=0)

```

#Handling Missing values

In general, with neural networks, it's safe to input missing values as 0, with the condition that 0 isn't already a meaningful value. The network will learn from exposure to the data that the value 0 means *missing data* and will start ignoring the value.

Note that if you're expecting missing values in the test data, but the network was trained on data without any missing values in the test data, the network won't have learned to ignore missing values!

#4.3.2 Feature engineering

*Feature engineering* is the process of using your own knowledge about the data and about the machine-learning algorithm at hand to make the algorithm work better by applying hardcoded transformations to the data before it goes into the model.



The essence of feature engineering is: making a problem easier by expressing it in a simpler way. Requiring some domain knowledge about the problem.

* Good features still allow you to solve problems more elegantly while using fewer resources. For instance, it would be ridiculous to solve the problem of reading a clock face using a convolutional neural network.

* Good features let you solve a problem with far less data. The ability of deep-learning models to learn features on their own relies on having lots of training data available; if you have only a few samples, then the information value in their features becomes critical.

#4.4 Overfitting and underfitting

*Optimization* refers to the process of adjusting a model to get the best performance possible on the training data (the *learning* in *machine learning*), whereas *generalization* refers to how well the trained model performs on data it has never seen before.

At the beginning of training, optimization and generalization are correlated: the lower the loss on the training data, the lower the loss on test data. While this is happening, your model is said to be *underfit:* there is still progress to be made; the network hasn't yet modeled all revelant patterns in the training data. As training continues, generalization and validation metrics stall while the network learns patterns specific to the training data.

To prevent a model from learning misleading or irrelevant patterns found in the training data, *the best solution is to get more training data*. A model trained on more data will naturally generalize better. If a network can only afford to memorize a small number of patterns, the optimization process will force it to focus on the most prominent patterns, which have a better chance of generalizing well.

The process of fighting overfitting this way is called *reguralization*.

#4.4.1 Reducing the network's size

The simplest way to prevent overfitting is to reduce the size of the model: the number of learnable parameters in the model( which is determined by the number of layers and the number of units per laye). In deep learning, the number of learnable parameters in a model is often referred to as the model's *capacity*. Intuitively, a model with more parameters has more *memorization capacity* and therefore can easily learn a perfect dictionary-like mapping between training samples and their targets--a mapping between training samples and their targets--a mapping without any generalization power.

On the other hand, if the network has limited memorization resources, it won't be ablle to learn this mapping as easily; thus, in order to minimize its loss, it will have to resort to learning compressed representations that have predictive power regarding the targets--precisely the type of representations we're interested in. At the same time, keep in mind that you should use models that have enough parameters that they don't underfit: your model shouldn't be starved for memorization resources. There is a compromise to be found between *too much capacity* and *not enough capacity*.

In [ ]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

Smaller network for movie-review classification network.

In [ ]:
model = model.Sequential()
model.add(layers.Dense(4, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(4, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

AttributeError: ignored

The smaller network starts overfitting later than the reference network, and its performance degrads more slowly once it starts overfitting.